# Imports

In [34]:
!pip install --upgrade pyalgotrading

In [35]:
from pyalgotrading.algobulls import AlgoBullsConnection

# Connection

In [36]:
connection = AlgoBullsConnection()

In [37]:
connection.get_token_url()

Please login to this URL to get your unique token: https://app.algobulls.com/settings?section=developerOptions


In [38]:
API_TOKEN = "c3477e22990b004e542ac6eaee59d3cc961816f5"
connection.set_access_token(API_TOKEN)

Access token is valid.


# Strategy Creation

## Import Strategy from pyaglostrategypool

In [39]:
! wget -O options_box_spread.py https://raw.githubusercontent.com/algobulls/pyalgostrategypool/master/pyalgostrategypool/options_box_spread/_strategy.py
! sed -i '1s/^/from pyalgotrading.strategy import StrategyBase\n/' options_box_spread.py

--2024-12-25 22:35:20--  https://raw.githubusercontent.com/Sriramg89/pyalgostrategypool/master/pyalgostrategypool/options_box_spread/_strategy.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9624 (9.4K) [text/plain]
Saving to: 'options_box_spread.py'

options_box_spread. 100%[===================>]   9.40K  --.-KB/s    in 0s      

2024-12-25 22:35:20 (21.9 MB/s) - 'options_box_spread.py' saved [9624/9624]

sed: 1: "options_box_spread.py": invalid command code o


In [40]:
from options_box_spread import OptionsBoxSpread as strategy_cls

In [41]:
response = connection.create_strategy(strategy_cls, overwrite=True)
response

{'message': 'Strategy saved successfully',
 'cstcId': 160105,
 'strategyId': 'c264033572fc423098c0e3aac739babb'}

In [42]:
strategy = response['strategyId']

# Strategy Testing

## Instruments Searching (optional)

In [43]:
instrument = connection.search_instrument('NIFTY 50', exchange='NSE')
instrument

[{'id': 61576, 'value': 'NSE:NIFTY 50'}]

## Setup Parameters

In [44]:
parameters = {
    # Leg 1
    'LEG_ONE_TRANSACTION_TYPE': 1,   # BUY
    'LEG_ONE_TRADING_SYMBOL_SUFFIX': 1,  # CE
    'LEG_ONE_STRIKE_DIRECTION': 1,   # ATM
    'LEG_ONE_NUMBER_OF_STRIKES': 1,  # 1 strike offset

    # Leg 2
    'LEG_TWO_TRANSACTION_TYPE': 2,   # SELL
    'LEG_TWO_TRADING_SYMBOL_SUFFIX': 1,  # CE
    'LEG_TWO_STRIKE_DIRECTION': 2,   # OTM
    'LEG_TWO_NUMBER_OF_STRIKES': 1,  # 1 strike offset

    # Leg 3
    'LEG_THREE_TRANSACTION_TYPE': 1,  # BUY
    'LEG_THREE_TRADING_SYMBOL_SUFFIX': 2,  # PE
    'LEG_THREE_STRIKE_DIRECTION': 2,  # OTM
    'LEG_THREE_NUMBER_OF_STRIKES': 1,  # 1 strike offset

    # Leg 4
    'LEG_FOUR_TRANSACTION_TYPE': 2,   # SELL
    'LEG_FOUR_TRADING_SYMBOL_SUFFIX': 2,  # PE
    'LEG_FOUR_STRIKE_DIRECTION': 1,   # ATM
    'LEG_FOUR_NUMBER_OF_STRIKES': 1,  # 1 strike offset

    # Arbitrage parameters
    'RISK_FREE_RATE': 0.01,
    'EXIT_THRESHOLD': 0.03,  
    'TRANSACTION_COST_PERCENT': 0.001 
}

In [45]:
initial_virtual_funds = 1500000     # in Rupees

## Backtesting

### Start

In [46]:
connection.backtest(
    strategy=strategy, 
    start='2019-08-01 09:15 +0530', 
    end='2021-07-31 15:30 +0530', 
    instrument='NSE:NIFTY 50', 
    lots=1,
    parameters=parameters, 
    candle='1 hour',
    initial_funds_virtual=initial_virtual_funds

)


Starting the strategy 'Options Box Spread' in BACKTESTING mode...
╒═════════════════════════╤════════════════════════════════════════╕
│ Config                  │ Value                                  │
╞═════════════════════════╪════════════════════════════════════════╡
│ Strategy Name           │ Options Box Spread                     │
├─────────────────────────┼────────────────────────────────────────┤
│ Trading Type            │ BACKTESTING                            │
├─────────────────────────┼────────────────────────────────────────┤
│ Instrument(s)           │ ['NSE:NIFTY 50']                       │
├─────────────────────────┼────────────────────────────────────────┤
│ Quantity/Lots           │ 1                                      │
├─────────────────────────┼────────────────────────────────────────┤
│ Start Timestamp         │ 2019-08-01 09:15:00+05:30              │
├─────────────────────────┼────────────────────────────────────────┤
│ End Timestamp           │ 2021-07-

### Status

In [47]:
connection.get_backtesting_job_status(strategy)

{'message': 'STARTING', 'status': 1}

### Logs

In [48]:
logs = connection.get_backtesting_logs(strategy)

Looking for a dedicated virtual server to execute your strategy... (3)

In [49]:
print(logs)

[2024-12-25 22:35:50] Performing sanity checks on cfg strategy_parameters, setting up required data structures...
[2024-12-25 22:35:50] Sanity checks on cfg successful.
[2024-12-25 22:35:50] Currency set to "INR"
[2024-12-25 22:35:50] Setting up broker connection...
[2024-12-25 22:35:51] Broker connection has been setup successfully.
[2024-12-25 22:35:51] (NSE_IDX) Funds available in client's ABVIRTUALBROKER account is : INR '1500000.00'
[2024-12-25 22:35:51] 
########################################
 INITIALIZING ALGOBULLS CORE (v3.3.0)... 
########################################
[2024-12-25 22:35:51] Welcome ALGOBULLS VIRTUAL USER!
[2024-12-25 22:35:51] Reading strategy...
[2024-12-25 22:35:51] Important: This class will be deprecated in future. Use StrategyOptionsBase class directly for all new strategies, irrespective of market type
[2024-12-25 22:35:51] Entering Backtesting mode. Henceforth, all timestamps will be Backtesting timestamps...
[BT] [2019-08-01 09:15:00,000] [INFO] [t

### Stop

In [50]:
connection.stop_backtesting_job(strategy)

Stopping BACKTESTING job... Success.


### Profit and Loss Reports

In [51]:
pnl_reports = connection.get_backtesting_report_pnl_table(strategy)
pnl_reports

,instrument_segment,instrument_tradingsymbol,entry_timestamp,entry_transaction_type,entry_quantity,entry_currency,entry_price,entry_variety,exit_timestamp,exit_transaction_type,exit_quantity,exit_currency,exit_price,exit_variety,pnl_absolute,net_pnl


### Statistics Reports

#### Statistics

In [ ]:
connection.get_backtesting_report_statistics(strategy, initial_funds=initial_virtual_funds)

#### Quantstats Full Report

In [ ]:
connection.get_backtesting_report_statistics(strategy, report='full', html_dump=True, initial_funds=initial_virtual_funds)

### Order History

In [ ]:
order_history = connection.get_backtesting_report_order_history(strategy)
print(order_history)

---

## Papertrading

### Start

In [ ]:
connection.papertrade(
    strategy=strategy, 
    start='9:15 +0530', 
    end='15:00 +0530',  
    instruments='NSE:NIFTY 50', 
    lots=5,
    parameters=parameters, 
    candle='60minutes',
)

### Status

In [ ]:
connection.get_papertrading_job_status(strategy)

### Logs

In [ ]:
logs = connection.get_papertrading_logs(strategy)
print(logs)

### Stop

In [ ]:
connection.stop_papertrading_job(strategy)

### Profit and Loss Reports (Paper Trading)

In [ ]:
pnl_reports = connection.get_papertrading_report_pnl_table(strategy)
pnl_reports

### Statistics Reports (Paper Trading)

#### Statistics

In [ ]:
connection.get_papertrading_report_statistics(strategy)

#### Quantstats Full Report

In [ ]:
connection.get_papertrading_report_statistics(strategy, mode='quantstats', report='full', html_dump=True)

### Order History (Paper Trading)

In [ ]:
order_history = connection.get_papertrading_report_order_history(strategy)
print(order_history)

---

## Live Trading (Real Trading)

### Start

In [ ]:
connection.realtrade(
    strategy=strategy, 
    start='8:00 +0530', 
    end='15:00 +0530', 
    instruments='NSE:SBIN', 
    lots=5,
    parameters=parameters, 
    candle='1 minute',
    broking_id=73,
    broking_details=broking_details
)

### Status

In [ ]:
connection.get_realtrading_job_status(strategy)

### Logs

In [ ]:
logs = connection.get_realtrading_logs(strategy)
print(logs)

### Stop

In [ ]:
connection.stop_realtrading_job(strategy)

### Profit and Loss Reports (Live Trading)

In [ ]:
pnl_reports = connection.get_realtrading_report_pnl_table(strategy)
pnl_reports

### Statistics Reports (Live Trading)

#### Statistics

In [ ]:
connection.get_realtrading_report_statistics(strategy)

#### Quantstats Full Report

In [ ]:
connection.get_realtrading_report_statistics(strategy, mode='quantstats', report='full', html_dump=True)